In [1]:
import torch
import torch.nn as nn
from torch import Tensor
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

In [2]:
with open("../Data/s_given_t_train.txt") as train_data:
    print(1)
    

1


In [3]:
embedding = nn.Embedding(10, 3)

In [6]:
input = Variable(torch.LongTensor([[1,2,4,5],[4,3,2]]))

RuntimeError: inconsistent sequence length at index (1, 3) - expected 4 but got 3

In [5]:
embedding(input)

Variable containing:
(0 ,.,.) = 
 -0.4424 -0.3841  0.7366
 -2.3084  0.3380  0.6742
 -0.0590  1.1383  0.7679
 -0.4721 -2.0669  0.1935

(1 ,.,.) = 
 -0.0590  1.1383  0.7679
 -0.0349 -0.0772 -1.4667
 -2.3084  0.3380  0.6742
 -0.3150 -0.3898 -0.1103
[torch.FloatTensor of size 2x4x3]